In [ ]:
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import preprocess_input
import numpy as np
import os

import sys
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.datasets import load_svmlight_file
from sklearn import preprocessing
import pylab as pl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import timeit
import csv
from itertools import zip_longest

#Definindo algumas variáveis

In [73]:
## Google Drive
drive_path = './'

## Arquivo de entrada
entrada = drive_path + 'test.txt'

arq = open(entrada,'r')
conteudo_entrada = arq.readlines()
arq.close()

## Diretorio da base de dados (imagens)
dir_dataset = drive_path + 'data/'

## Diretorio de destino
dir_destino = drive_path + 'libsvm/'

### Cria o diretorio de destino (caso nao exista)
# if not os.path.exists(dir_destino):
# 	os.makedirs(dir_destino)

# Cria arquivo com as caracteristicas
arq_svm = dir_destino + 'out.svm'
file_svm = open(arq_svm, 'w')

# Input resize (minimo=75x75)
img_rows, img_cols = 100, 100

print('Done')

Done


#Importando modelo pré-treinado

In [ ]:
# InceptionV3
# - weights='imagenet' (inicializa pesos pre-treinado na ImageNet)
# - include_top=False (nao inclui as fully-connected layers)
# - input_shape=(299, 299, 3) (DEFAULT) (minimo=75x75)
model = VGG16(weights='imagenet', include_top=False)

# Mostra a arquitetura da rede
model.summary()


#Processando as imagens e salvando no formato LIBSVM

In [74]:
print ("Loading...")

for i in conteudo_entrada:
  
  nome, classe = i.split()

  img_path = dir_dataset + nome
  print (img_path) ##
  
  img = image.load_img(img_path, target_size=(img_rows,img_cols))
  img_data = image.img_to_array(img)
  img_data = np.expand_dims(img_data, axis=0)
  img_data = preprocess_input(img_data)

  # Passa a imagem pela rede
  inception_features = model.predict(img_data)

  # Flatten
  features_np = np.array(inception_features)
  features_np = features_np.flatten()

  # Salva no formato do libsvm
  file_svm.write(classe+' ')
  for j in range (features_np.size):
    file_svm.write(str(j+1)+':'+str(features_np[j])+' ')
  file_svm.write('\n')

print (features_np.size)
file_svm.close()

Loading...
./data/AD1328.jpg
./data/AD1329.jpg
./data/AD1330.jpg
./data/AD1331.jpg
./data/AD1332.jpg
./data/AD1333.jpg
./data/AD1334.jpg
./data/AD1335.jpg
./data/AD1336.jpg
./data/AD1337.jpg
./data/AD1338.jpg
./data/AD1468.jpg
./data/AD1480.jpg
./data/AD1492.jpg
./data/AD1504.jpg
./data/AD1516.jpg
./data/AD1528.jpg
./data/AD1540.jpg
./data/AD1551.jpg
./data/AD1552.jpg
./data/AD1553.jpg
./data/AD1563.jpg
./data/AD1564.jpg
./data/AD1565.jpg
./data/AD1591.jpg
./data/AD1592.jpg
./data/AD1593.jpg
./data/AD1594.jpg
./data/AD1598.jpg
./data/AD1599.jpg
./data/td0029.jpg
./data/td0030.jpg
./data/td0031.jpg
./data/td0054.jpg
./data/td0055.jpg
./data/td0056.jpg
./data/td0057.jpg
./data/td0058.jpg
./data/td0059.jpg
./data/td0086.jpg
./data/td0087.jpg
./data/td0088.jpg
./data/td0089.jpg
./data/td0090.jpg
./data/td0111.jpg
./data/td0112.jpg
./data/td0113.jpg
./data/td0114.jpg
./data/td0116.jpg
./data/td0117.jpg
./data/td0118.jpg
./data/td0119.jpg
./data/td0120.jpg
./data/td0139.jpg
./data/td0140.jpg

In [75]:
def kNN (train, test, K_value, distance):
    #carrega os dados da base de treinamento
    x_train, y_train = load_svmlight_file(train)
    # x_train = x_data[0:train_block]
    # y_train = y_data[0:train_block]

    #carrega os dados da base de teste
    x_test, y_test = load_svmlight_file(test)


    #separa o bloco de dados a ser utilizado
    # if (train_block != 20000):
    #     x_train, x_discard, y_train, y_discard = train_test_split(x_data, y_data, train_size=train_block, random_state = 5)
    # else:
    #     x_train = x_data
    #     y_train = y_data

    
    # x vetor de características e Y labels
    x_train = x_train.toarray()
    x_test = x_test.toarray()


    #criar um kNN
    neigh = KNeighborsClassifier(n_neighbors=K_value, metric=distance)


    #treinamento
    neigh.fit(x_train, y_train)
    
    y_pred = neigh.predict(x_test)

    # mostra o resultado do classificador na base de teste
    print ('[KNN] Accuracy: ',  neigh.score(x_test, y_test))
        
        
    # cria a matriz de confusao e salva no vetor de confusões
    cm = confusion_matrix(y_test, y_pred)
    print ('[KNN] Confusion Matrix: \n', cm)


In [76]:
def perceptron (train, test, tolerance, iterations):
    #carrega os dados da base de treinamento
    x_train, y_train = load_svmlight_file(train)

    #carrega os dados da base de teste
    x_test, y_test = load_svmlight_file(test)


    #separa o bloco de dados a ser utilizado
    # if (train_block != 20000):
    #     x_train, x_discard, y_train, y_discard = train_test_split(x_data, y_data, train_size=train_block, random_state = 5)
    # else:
    #     x_train = x_data
    #     y_train = y_data

    
    # x vetor de características e Y labels
    x_train = x_train.toarray()
    x_test = x_test.toarray()


    #criar um perceptron
    percep = Perceptron(tol=tolerance, max_iter=iterations, random_state=random.randint(0, 42), shuffle=True, n_jobs=4, n_iter_no_change=10)

    #treinamento
    percep.fit(x_train, y_train)


    y_pred = percep.predict(x_test)

    # mostra o resultado do classificador na base de teste
    print ('[Perceptron] Accuracy: ',  percep.score(x_test, y_test))
        
        
    # cria a matriz de confusao e salva no vetor de confusões
    cm = confusion_matrix(y_test, y_pred)
    print ('[Perceptron] Confusion Matrix: \n', cm)

In [77]:
def svm (train, test):
    #carrega os dados da base de treinamento
    x_train, y_train = load_svmlight_file(train)

    #carrega os dados da base de teste
    x_test, y_test = load_svmlight_file(test)


    #separa o bloco de dados a ser utilizado
    # if (train_block != 20000):
    #     x_train, x_discard, y_train, y_discard = train_test_split(x_data, y_data, train_size=train_block, random_state = 5)
    # else:
    #     x_train = x_data
    #     y_train = y_data

    
    # x vetor de características e Y labels
    x_train = x_train.toarray()
    x_test = x_test.toarray()


    #criar um svm
    clf = SVC(kernel='linear')

    #treinamento
    clf.fit(x_train, y_train)


    y_pred = clf.predict(x_test)

    # mostra o resultado do classificador na base de teste
    print ('[SVM] Accuracy: ',  clf.score(x_test, y_test))
        
        
    # cria a matriz de confusao e salva no vetor de confusões
    cm = confusion_matrix(y_test, y_pred)
    print ('[SVM] Confusion Matrix: \n', cm)

In [79]:
train_archive = './libsvm/train.svm'
test_archive = './libsvm/test.svm'

kNN(train_archive, test_archive, 9,'euclidean')

[KNN] Accuracy:  0.770573566084788
[KNN] Confusion Matrix: 
 [[29  5  0  0  0  2  2  0  1  0  0  0]
 [ 6 20  0  0  1  0  0  0  0  1  0  4]
 [ 0  0 35  0  1  0  0  0  0  0  0  0]
 [ 1  0  0 35  2  0  0  1  0  0  0  0]
 [ 3  0  1  2 31  0  1  0  0  0  0  0]
 [ 2  0  0  0  0 19  6  0  1  0  0  1]
 [ 6  0  0  1  1  6 17  0  0  0  0  1]
 [ 1  0  2  0  0  1  0 24  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 24  3  1  3]
 [ 0  0  0  3  0  0  0  0  3 24  0  0]
 [ 0  0  0  0  0  0  0  0  2  1 31  0]
 [ 0  1  0  2  0  2  0  1  4  2  1 20]]


In [80]:
train_archive = './libsvm/train.svm'
test_archive = './libsvm/test.svm'

perceptron(train_archive, test_archive, 0.0001, 100)

[Perceptron] Accuracy:  0.85785536159601
[Perceptron] Confusion Matrix: 
 [[33  0  0  0  0  4  0  1  1  0  0  0]
 [ 6 21  0  1  1  0  0  0  0  0  0  3]
 [ 0  0 36  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 36  0  0  1  0  0  0  0  2]
 [ 0  0  0  0 35  1  1  0  0  0  1  0]
 [ 2  0  0  0  0 25  2  0  0  0  0  0]
 [ 1  0  0  0  0  9 21  1  0  0  0  0]
 [ 0  0  0  0  0  0  0 28  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 25  0  0  6]
 [ 0  0  0  3  0  0  0  0  0 27  0  0]
 [ 0  0  1  0  0  0  0  0  0  0 32  1]
 [ 0  1  0  1  0  2  0  1  2  0  1 25]]


In [81]:
train_archive = './libsvm/train.svm'
test_archive = './libsvm/test.svm'

svm(train_archive, test_archive)

[SVM] Accuracy:  0.8728179551122195
[SVM] Confusion Matrix: 
 [[34  3  0  0  0  2  0  0  0  0  0  0]
 [ 4 24  0  0  1  0  0  0  0  0  0  3]
 [ 0  0 35  0  1  0  0  0  0  0  0  0]
 [ 1  0  0 36  1  0  0  0  0  0  0  1]
 [ 0  1  0  3 33  0  1  0  0  0  0  0]
 [ 2  0  0  0  0 25  2  0  0  0  0  0]
 [ 0  0  0  0  0  6 26  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 28  0  0  0  0]
 [ 0  1  0  0  0  0  0  0 26  2  0  2]
 [ 0  0  0  0  0  0  0  0  1 29  0  0]
 [ 0  0  1  0  0  0  0  0  0  0 33  0]
 [ 1  1  0  2  1  0  0  3  3  0  1 21]]
